# *Import Libraries*

In [1]:
import pandas as pd
import datetime as dt
import geopandas as gpd

# *Load Spreadsheet*
Data is loaded from kawal Covid19:
- https://docs.google.com/spreadsheets/d/1ma1T9hWbec1pXlwZ89WakRk-OfVUQZsOCFl4FwZxzVw/htmlview
- https://datawrapper.dwcdn.net/MwHOx/247/
- https://datawrapper.dwcdn.net/BA77E/183/
- https://datawrapper.dwcdn.net/0p5op/157/

In [2]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request


def get_table(area):
    # If modifying these scopes, delete the file token.pickle.
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

    # The ID and range of a sample spreadsheet.
    SAMPLE_SPREADSHEET_ID = '1zvIBRuuumGSf69v8aiDTxSrRg5t8OVcT-CGkuCPiFJY'
    SAMPLE_RANGE_NAME = area
    
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)


    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range=SAMPLE_RANGE_NAME).execute()
    values = result.get('values', [])
    
    return values

Since the data are collected from several sources, the name of provinces are not consitent, we need to make them consistent.

In [3]:
abbr = {
    'Aceh' : 'Aceh',
    'Bali' : 'Bali',
    'Banten' : 'Banten',
    'Bangka Belitung' : 'Babel',
    'Bangka-Belitung' : 'Babel',
    'Bengkulu' : 'Bengkulu',
    'DI Yogyakarta' : 'DIY',
    'DKI Jakarta' : 'Jakarta',
    'Jakarta Raya': 'Jakarta',
    'Gorontalo' : 'Gorontalo',
    'Jambi' : 'Jambi', 
    'Jawa Barat' : 'Jabar', 
    'Jawa Tengah' : 'Jateng', 
    'Jawa Timur' : 'Jatim', 
    'Kalimantan Barat' : 'Kalbar', 
    'Kalimantan Tengah' : 'Kalteng', 
    'Kalimantan Timur' : 'Kaltim',
    'Kalimantan Selatan' : 'Kalsel', 
    'Kalimantan Utara' : 'Kaltara', 
    'Kep Riau' : 'Kepri',
    'Kepulauan Riau' : 'Kepri',
    'Lampung' : 'Lampung', 
    'Maluku' : 'Maluku', 
    'Maluku Utara' : 'Malut', 
    'Nusa Tenggara Barat' : 'NTB', 
    'Nusa Tenggara Timur' : 'NTT', 
    'Papua' : 'Papua', 
    'Papua Barat' : 'Papbar', 
    'Irian Jaya Barat' : 'Papbar',
    'Sulawesi Barat' : 'Sulbar', 
    'Riau' : 'Riau', 
    'Sulawesi Selatan' : 'Sulsel', 
    'Sulawesi Tengah' : 'Sulteng', 
    'Sulawesi Tenggara' : 'Sultra', 
    'Sulawesi Utara' : 'Sulut', 
    'Sumatera Barat' : 'Sumbar',
    'Sumatera Selatan' : 'Sumsel', 
    'Sumatera Utara' : 'Sumut'
}

In [4]:
# Daily statistics
values = get_table('Statistik Harian!A1:AJ244')
df_statistics = pd.DataFrame(values[1:], columns=values[0])
df_statistics.rename(columns={'':'Date'}, inplace=True)

date_list = pd.date_range(start='03/02/2020', end='31/10/2020', freq='D')
date_list = date_list.drop(dt.datetime.strptime('2020-03-11', '%Y-%m-%d'))
df_statistics['Date'] = date_list

outfile = open('files/df_statistics', 'wb')
pickle.dump(df_statistics, outfile)
outfile.close()

df_statistics

,Date,Kasus baru,Kasus Impor,Kasus Lokal,Total kasus,Kasus aktif,% kasus aktif,Sembuh\n(baru),Sembuh,Tingkat kesembuhan (seluruh kasus),...,Test/Positif,Jumlah org yg dites / positif,Jumlah Test/Positif harian,Positif / Jumlah Tes,Positive rate harian,Case Growth Rate,Positive rate mingguan,Jumlah spesimen diperiksa (rata-rata 7 hari),Jumlah orang diperiksa (rata-rata 7 hari),Rasio spesimen/orang diperiksa (rata-rata 7 hari)
0,2020-03-02,2,0,2,2,2,100.00%,0,0,0.00%,...,169.50,169.50,0.00,0.59%,None,None,None,None,None,None
1,2020-03-03,0,0,0,2,2,100.00%,0,0,0.00%,...,170.50,170.50,#DIV/0!,0.59%,0.00%,0.00%,None,None,None,None
2,2020-03-04,0,0,0,2,2,100.00%,0,0,0.00%,...,186.00,186.00,#DIV/0!,0.54%,0.00%,0.00%,None,None,None,None
3,2020-03-05,0,0,0,2,2,100.00%,0,0,0.00%,...,194.00,194.00,#DIV/0!,0.52%,0.00%,0.00%,None,None,None,None
4,2020-03-06,2,0,2,4,4,100.00%,0,0,0.00%,...,112.50,112.50,31.00,0.89%,3.77%,100.00%,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,2020-10-27,3520,,,396454,"60,694",15.31%,"4,576","322,248",81.28%,...,11.07,7.01,8.15,14.27%,12.26%,0.90%,14.17%,"37,910","27,841",1.36
239,2020-10-28,4029,,,400483,"61,078",15.25%,"3,545","325,793",81.35%,...,11.06,7.00,6.79,14.28%,14.73%,1.02%,14.28%,"37,480","27,376",1.37
240,2020-10-29,3565,,,404048,"60,569",14.99%,"3,985","329,778",81.62%,...,11.05,7.01,7.12,14.27%,14.04%,0.89%,14.44%,"36,107","26,230",1.38
241,2020-10-30,2897,,,406945,"58,868",14.47%,"4,517","334,295",82.15%,...,11.03,7.01,8.04,14.26%,12.45%,0.72%,14.37%,"33,616","24,887",1.35


In [5]:
# Province summary
values = get_table('Kasus per Provinsi!A2:BC36')
df_province_summary = pd.DataFrame(values[1:], columns=values[0])
df_province_summary = df_province_summary.iloc[:,[18,21,25,31,35,39,40,41,42,43,44,45,46,47,48,49,50,51]]
df_province_summary.iloc[:,1:] = df_province_summary.iloc[:,1:].replace(',','', regex=True).apply(pd.to_numeric).fillna(0)
df_province_summary.set_index('Provinsi Asal', inplace=True)
df_province_summary.rename(columns={'Covid19+ \nMeninggal':'Covid19+ Meninggal'}, index=abbr, inplace=True)

outfile = open('files/df_province_summary', 'wb')
pickle.dump(df_province_summary, outfile)
outfile.close()
df_province_summary.to_csv('files/df_province_summary.csv', sep=';')

df_province_summary

,ODP Total,PDP,Suspect Total,Probable Total,Rapid Test,Kasus (+) Rapid Test,Tes PCR (Sampel),Tes PCR (Sampel +),Tes PCR (Sampel -),Tes PCR (Kasus Baru),Result (+),Result (-),Covid19+ Total,Covid19+ Dirawat,Covid19+ Sembuh,Covid19+ Meninggal,Covid19+ Isolasi
Provinsi Asal,,,,,,,,,,,,,,,,,
Aceh,0,0,3544,450,0,0,0,0,0,0,0,0,7426,1504,5650,272,0
Bali,0,0,0,0,0,0,0,0,0,0,0,0,11807,737,10681,389,0
Banten,748,188,20460,40,0,0,0,0,0,0,0,0,9425,1190,7940,295,0
Babel,0,2066,0,0,0,0,0,0,0,6163,587,5576,587,54,526,7,0
Bengkulu,3258,0,0,0,0,0,0,0,0,10125,1082,9043,1082,277,755,50,0
DIY,1507,212,13948,3975,0,0,0,0,0,0,0,0,3851,598,3160,93,0
Jakarta,0,0,185855,3961,573873,26321,1.69858e+06,231154,1.46743e+06,1.28852e+06,106205,1.18231e+06,106205,2178,94819,2273,6935
Jambi,0,0,3309,0,0,0,8228,1230,6998,0,0,0,1230,588,619,23,0
Jabar,0,0,84211,1650,363033,7351,549971,53919,491493,351173,40403,308743,37710,5881,30459,1034,336


In [6]:
# Age group
values = get_table('Tingkat Kematian!A4:L10')
df_age = pd.DataFrame(values[1:], columns=values[0])
df_age.iloc[:,[2,4,7,10]] = df_age.iloc[:,[2,4,7,10]].replace(',','', regex=True).apply(pd.to_numeric).fillna(0)

outfile = open('files/df_age', 'wb')
pickle.dump(df_age, outfile)
outfile.close()

df_age

,Kel. Usia,Proporsi,Kasus dg data usia,Proporsi,Sembuh dg data usia,% sembuh,Proporsi,Dirawat dg data usia,% dirawat,Proporsi,Kematian dg data usia,Tingkat kematian
0,0-5,2.50%,9525,2.60%,8060,84.62%,2.70%,1361,14.29%,0.80%,104,1.09%
1,6-17,8.10%,30862,8.30%,25730,83.37%,6.40%,5015,16.25%,0.90%,117,0.38%
2,18-30,24.80%,94490,25.50%,79051,83.66%,22.00%,14972,15.84%,3.60%,467,0.49%
3,31-45,30.90%,117731,31.60%,97961,83.21%,30.00%,18069,15.35%,13.10%,1701,1.44%
4,46-59,23.40%,89156,22.90%,70991,79.63%,26.30%,13153,14.75%,38.60%,5012,5.62%
5,≥60,10.40%,39625,9.10%,28210,71.19%,12.60%,5818,14.68%,43.10%,5597,14.12%


In [7]:
# PCR
values = get_table('Copy of Data PCR - Kemenkes!B2:W36')
df_pcr = pd.DataFrame(values[1:], columns=values[0])
df_pcr = df_pcr.iloc[:,[0,2,9,10,11,12,13,14,15,16,17,18,19,20,21]]
df_pcr.iloc[:,1:12] = df_pcr.iloc[:,1:12].replace(',','', regex=True).apply(pd.to_numeric).fillna(0)
df_pcr.set_index('Provinsi Asal', inplace=True)
df_pcr.rename(index=abbr, inplace=True)

outfile = open('files/df_pcr', 'wb')
pickle.dump(df_pcr, outfile)
outfile.close()

df_pcr

,Jumlah Pemeriksaan Sample PCR,suspect,Jumlah PCR+TCM,Penduduk,Probable,Negatif,PR+TCM/1 jt,T,P,S,MD,Check sum,Positivity Rate,Testing Ratio
Provinsi Asal,,,,,,,,,,,,,,
Aceh,1265,2381,2295,5459891,0,1729,420,79,52,25,2,ok,4.37%,22.9
Bali,20130,3927,32378,4380824,802,22628,7391,1414,630,771,13,ok,5.88%,17.0
Banten,18283,1224,4968,1517590,7,4013,3274,149,16,131,2,ok,3.58%,27.9
Babel,3139,12701,25096,13160496,2486,13599,1907,1438,731,628,79,ok,9.56%,10.5
Bengkulu,739,1009,1317,2019848,17,485,652,125,26,89,10,ok,20.49%,4.9
DIY,10706,9379,15513,3882288,114,9717,3996,306,38,260,8,ok,3.05%,32.8
Jakarta,160240,104597,232848,10644986,1134,154048,21874,11114,4624,5865,625,ok,6.73%,14.9
Jambi,330,3930,5095,1219576,35,4171,4178,243,40,195,8,ok,5.51%,18.2
Jabar,53600,2603,330,3677894,7,0,90,117,48,69,0,ok,100.00%,1.0


In [8]:
# Timeline total case each province
values = get_table('Timeline!A1:AI230')
df_total_case = pd.DataFrame(values[1:], columns=values[0])
df_total_case.iloc[:,1:] = df_total_case.iloc[:,1:].replace(',','', regex=True).apply(pd.to_numeric).fillna(0)
df_total_case.rename(columns=abbr, inplace=True)

df_total_case.rename(columns={'Total Kasus':'Date'}, inplace=True)
df_total_case['Date'] = pd.date_range(start='03/18/2020', end='11/01/2020', freq='D')

outfile = open('files/df_total_case', 'wb')
pickle.dump(df_total_case, outfile)
outfile.close()

df_total_case

,Date,Aceh,Bali,Banten,Babel,Bengkulu,DIY,Jakarta,Jambi,Jabar,...,Sulteng,Lampung,Riau,Malut,Maluku,Papbar,Papua,Sulbar,NTT,Gorontalo
0,2020-03-18,0,1,17,0,0,3,158,0,24,...,0,1,1,0,0,0,0,0,0,0
1,2020-03-19,0,1,27,0,0,5,210,0,26,...,0,1,2,0,0,0,0,0,0,0
2,2020-03-20,0,4,37,0,0,4,215,0,41,...,0,1,1,0,0,0,0,0,0,0
3,2020-03-21,0,3,43,0,0,5,267,0,55,...,0,1,1,0,0,0,0,0,0,0
4,2020-03-22,0,3,47,0,0,5,307,0,59,...,0,1,1,0,1,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,2020-10-28,7339,11588,9208,573,1040,3662,103522,1188,35148,...,848,1729,14251,2196,3785,4080,8910,1000,669,3007
225,2020-10-29,7373,11647,9299,578,1049,3744,104235,1219,35607,...,855,1757,14511,2196,3790,4088,8930,1000,679,3007
226,2020-10-30,7394,11712,9406,581,1081,3803,104847,1230,35927,...,855,1794,14672,2199,3790,4136,8992,1027,679,3017
227,2020-10-31,7405,11764,9490,583,1081,3835,105597,1230,36338,...,857,1825,14799,2199,3852,4149,9037,1029,681,3017


In [9]:
# Timeline daily case each province
values = get_table('Timeline!A232:AI464')
df_daily_case = pd.DataFrame(values[1:], columns=values[0])
df_daily_case = df_daily_case.fillna(0).replace('', 0)
df_daily_case.iloc[:,1:] = df_daily_case.iloc[:,1:].replace(',','', regex=True).apply(pd.to_numeric).fillna(0)
df_daily_case.rename(columns=abbr, inplace = True)

df_daily_case.rename(columns={'Kasus Harian':'Date'}, inplace=True)
df_daily_case['Date'] = pd.date_range(start='03/15/2020', end='11/01/2020', freq='D')

outfile = open('files/df_daily_case', 'wb')
pickle.dump(df_daily_case, outfile)
outfile.close()

df_daily_case

,Date,Aceh,Bali,Banten,Babel,Bengkulu,DIY,Jakarta,Jambi,Jabar,...,Sulteng,Lampung,Riau,Malut,Maluku,Papbar,Papua,Sulbar,NTT,Gorontalo
0,2020-03-15,0,0,0,0,0,0,19,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-03-16,0,0,1,0,0,0,14,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2020-03-17,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-03-18,0,0,4,0,0,1,30,0,12,...,0,1,1,0,0,0,0,0,0,0
4,2020-03-19,0,0,10,0,0,2,52,0,2,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,2020-10-28,74,68,170,7,23,45,844,32,403,...,10,43,253,3,13,106,125,3,2,2
228,2020-10-29,34,59,91,5,9,82,713,31,459,...,7,28,260,0,5,8,20,0,10,0
229,2020-10-30,21,65,107,3,32,59,612,11,320,...,0,37,161,3,0,48,62,27,0,10
230,2020-10-31,11,52,84,2,0,32,750,0,411,...,2,31,127,0,62,13,45,2,2,0


In [10]:
# Timeline active case each province
values = get_table('Timeline!A466:AI692')
df_active_case = pd.DataFrame(values[1:], columns=values[0])
df_active_case = df_active_case.fillna(0).replace('', 0)
df_active_case.iloc[:,1:] = df_active_case.iloc[:,1:].replace(',','', regex=True).apply(pd.to_numeric).fillna(0)
df_active_case.rename(columns=abbr, inplace=True)

df_active_case.rename(columns={'Kasus Aktif':'Date'}, inplace=True)
df_active_case['Date'] = pd.date_range(start='03/21/2020', end='11/01/2020', freq='D')

outfile = open('files/df_active_case', 'wb')
pickle.dump(df_active_case, outfile)
outfile.close()

df_active_case

,Date,Aceh,Bali,Banten,Babel,Bengkulu,DIY,Jakarta,Jambi,Jabar,...,Sulteng,Lampung,Riau,Malut,Maluku,Papbar,Papua,Sulbar,NTT,Gorontalo
0,2020-03-21,0,2,40,0,0,4,227,0,47,...,0,1,1,0,0,0,0,0,0,0
1,2020-03-22,0,1,43,0,0,4,256,0,45,...,0,1,1,0,1,0,2,0,0,0
2,2020-03-23,0,4,52,0,0,4,301,1,45,...,0,1,1,1,1,0,2,0,0,0
3,2020-03-24,0,4,60,0,0,4,370,1,45,...,0,1,2,1,1,0,3,0,0,0
4,2020-03-25,0,7,62,0,0,14,409,1,58,...,0,1,1,1,1,0,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,2020-10-28,2036,811,1916,56,204,559,11254,591,9927,...,201,682,3465,193,580,537,4144,170,199,83
222,2020-10-29,2070,812,1927,59,208,589,10875,603,9894,...,197,661,3498,193,557,530,4164,170,207,77
223,2020-10-30,2088,766,1850,62,230,587,10396,600,9854,...,193,673,3404,196,557,577,4226,190,174,85
224,2020-10-31,2098,853,1799,52,229,595,10095,588,9975,...,176,702,3301,196,573,483,4271,189,176,80


In [11]:
# Timeline total recovered each province
values = get_table('Timeline!A694:AI920')
df_total_recovered = pd.DataFrame(values[1:], columns=values[0])
df_total_recovered = df_total_recovered.fillna(0).replace('', 0)
df_total_recovered.iloc[:,1:] = df_total_recovered.iloc[:,1:].replace(',','', regex=True).apply(pd.to_numeric).fillna(0)
df_total_recovered.rename(columns=abbr, inplace=True)

df_total_recovered.rename(columns={'Sembuh':'Date'}, inplace=True)
df_total_recovered['Date'] = pd.date_range(start='03/21/2020', end='11/01/2020', freq='D')

outfile = open('files/df_total_recovered', 'wb')
pickle.dump(df_total_recovered, outfile)
outfile.close()

df_total_recovered

,Date,Aceh,Bali,Banten,Babel,Bengkulu,DIY,Jakarta,Jambi,Jabar,...,Sulteng,Lampung,Riau,Malut,Maluku,Papbar,Papua,Sulbar,NTT,Gorontalo
0,2020-03-21,0,0,1,0,0,1,17,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2020-03-22,0,0,1,0,0,1,22,0,5,...,0,0,0,0,0,0,0,0,0,0
2,2020-03-23,0,0,1,0,0,1,23,0,5,...,0,0,0,0,0,0,0,0,0,0
3,2020-03-24,0,0,1,0,0,1,23,0,5,...,0,0,0,0,0,0,0,0,0,0
4,2020-03-25,0,0,1,0,0,1,23,0,5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,2020-10-28,5037,10397,7030,510,788,3013,90064,574,24507,...,613,977,10467,1928,3158,3478,4637,816,463,2839
222,2020-10-29,5037,10453,7108,512,793,3064,91142,593,24999,...,624,1023,10688,1928,3186,3493,4637,816,465,2845
223,2020-10-30,5037,10561,7290,512,803,3124,92219,607,25355,...,628,1047,10937,1928,3186,3494,4637,823,498,2847
224,2020-10-31,5037,10624,7422,524,803,3147,93251,619,25644,...,647,1047,11162,1928,3232,3601,4637,826,498,2852


In [12]:
# Timeline daily recovered each province
values = get_table('Timeline!A922:AI1148')
df_daily_recovered = pd.DataFrame(values[1:], columns=values[0])
df_daily_recovered = df_daily_recovered.fillna(0).replace('', 0)
df_daily_recovered.iloc[:,1:] = df_daily_recovered.iloc[:,1:].replace(',','', regex=True).apply(pd.to_numeric).fillna(0)
df_daily_recovered.rename(columns=abbr, inplace=True)

df_daily_recovered.rename(columns={'Sembuh Harian':'Date'}, inplace=True)
df_total_recovered['Date'] = pd.date_range(start='03/21/2020', end='11/01/2020', freq='D')

outfile = open('files/df_daily_recovered', 'wb')
pickle.dump(df_daily_recovered, outfile)
outfile.close()

df_daily_recovered

,Date,Aceh,Bali,Banten,Babel,Bengkulu,DIY,Jakarta,Jambi,Jabar,...,Sulteng,Lampung,Riau,Malut,Maluku,Papbar,Papua,Sulbar,NTT,Gorontalo
0,21-Mar,0,0,0,0,0,1,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,22-Mar,0,0,0,0,0,0,5,0,4,...,0,0,0,0,0,0,0,0,0,0
2,23-Mar,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,24-Mar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,25-Mar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,28 Oct,0,36,94,15,3,42,1047,32,367,...,10,36,264,0,38,120,39,0,14,0
222,29 Oct,0,56,78,2,5,51,1078,19,492,...,11,46,221,0,28,15,0,0,2,6
223,30 Oct,0,108,182,0,10,60,1077,14,356,...,4,24,249,0,0,1,0,7,33,2
224,31 Oct,0,63,132,12,0,23,1032,12,289,...,19,0,225,0,46,107,0,3,0,5


In [13]:
# Timeline total death each province
values = get_table('Timeline!A1150:AI1379')
df_total_death = pd.DataFrame(values[1:], columns=values[0])
df_total_death = df_total_death.fillna(0).replace('', 0)
df_total_death.iloc[:,1:] = df_total_death.iloc[:,1:].replace(',','', regex=True).apply(pd.to_numeric).fillna(0)
df_total_death.rename(columns=abbr, inplace=True)

df_total_death.rename(columns={'Meninggal Dunia':'Date'}, inplace=True)
df_total_death['Date'] = pd.date_range(start='03/18/2020', end='11/01/2020', freq='D')

outfile = open('files/df_total_death', 'wb')
pickle.dump(df_total_death, outfile)
outfile.close()

df_total_death

,Date,Aceh,Bali,Banten,Babel,Bengkulu,DIY,Jakarta,Jambi,Jabar,...,Sulteng,Lampung,Riau,Malut,Maluku,Papbar,Papua,Sulbar,NTT,Gorontalo
0,2020-03-18,0,1,1,0,0,0,12,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2020-03-19,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2020-03-20,0,1,1,0,0,0,18,0,7,...,0,0,0,0,0,0,0,0,0,0
3,2020-03-21,0,1,2,0,0,0,23,0,7,...,0,0,0,0,0,0,0,0,0,0
4,2020-03-22,0,2,3,0,0,0,29,0,9,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,2020-10-28,266,380,262,7,48,90,2204,23,714,...,34,70,319,75,47,65,129,14,7,85
225,2020-10-29,266,382,264,7,48,91,2218,23,714,...,34,73,325,75,47,65,129,14,7,85
226,2020-10-30,269,385,266,7,48,92,2232,23,718,...,34,74,331,75,47,65,129,14,7,85
227,2020-10-31,270,287,269,7,49,93,2251,23,719,...,34,76,336,75,47,65,129,14,7,85


In [14]:
# Timeline daily death each province
values = get_table('Timeline!A1381:AI1606')
df_daily_death = pd.DataFrame(values[1:], columns=values[0])
df_daily_death = df_daily_death.fillna(0).replace('', 0)
df_daily_death.iloc[:,1:] = df_daily_death.iloc[:,1:].replace(',','', regex=True).apply(pd.to_numeric).fillna(0)
df_daily_death.rename(columns=abbr, inplace=True)

df_daily_death.rename(columns={'Meninggal Dunia Harian':'Date'}, inplace=True)
date_list = pd.date_range(start='03/21/2020', end='11/01/2020', freq='D')
date_list = date_list.drop(dt.datetime.strptime('2020-08-19', '%Y-%m-%d'))
df_daily_death['Date'] = date_list

outfile = open('files/df_daily_death', 'wb')
pickle.dump(df_daily_death, outfile)
outfile.close()

df_daily_death

,Date,Aceh,Bali,Banten,Babel,Bengkulu,DIY,Jakarta,Jambi,Jabar,...,Sulteng,Lampung,Riau,Malut,Maluku,Papbar,Papua,Sulbar,NTT,Gorontalo
0,2020-03-21,0,0,1,0,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-03-22,0,1,1,0,0,0,6,0,2,...,0,0,0,0,0,0,0,0,0,0
2,2020-03-23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2020-03-24,0,0,1,0,0,1,2,0,1,...,0,0,0,0,0,0,0,0,0,0
4,2020-03-25,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,2020-10-28,6,2,1,0,0,1,16,1,6,...,1,2,6,1,0,2,5,2,0,0
221,2020-10-29,0,2,2,0,0,1,14,0,0,...,0,3,6,0,0,0,0,0,0,0
222,2020-10-30,3,3,2,0,0,1,14,0,4,...,0,1,6,0,0,0,0,0,0,0
223,2020-10-31,1,2,3,0,1,1,19,0,1,...,0,2,5,0,0,0,0,0,0,0


# *Other sources*
- Commute index: https://sirusa.bps.go.id/sirusa/index.php/dasar/view?kd=1545&th=2019 (page 56,72)
- PCR: https://databoks.katadata.co.id/datapublish/2020/09/30/rasio-tes-pcr-per-1-juta-penduduk-paling-tinggi-di-jakarta# (29 September 2020)

# *LOAD MAP FILE*
- Source: http://www.diva-gis.org/datadown
- Remove unnecessary columns
- Rename columns
- Rename province names
- Set province as index

In [15]:
df_geo = gpd.read_file('files/IDN_adm1.shp')
df_geo.drop(['ID_0', 'ISO', 'NAME_0', 'ID_1', 'TYPE_1', 'ENGTYPE_1', 'NL_NAME_1', 'VARNAME_1'], axis=1, inplace=True)
df_geo.rename(columns={'NAME_1':'province'}, inplace=True)
df_geo.replace(abbr, inplace=True)
df_geo.set_index('province', inplace=True)

outfile = open('files/df_geo', 'wb')
pickle.dump(df_geo, outfile)
outfile.close()

df_geo

,geometry
province,
Aceh,"MULTIPOLYGON (((97.38493 1.97713, 97.38396 1.9..."
Bali,"MULTIPOLYGON (((115.62988 -8.77599, 115.63037 ..."
Babel,"MULTIPOLYGON (((108.06951 -3.80305, 108.06826 ..."
Banten,"MULTIPOLYGON (((105.54861 -6.99472, 105.54889 ..."
Bengkulu,"MULTIPOLYGON (((102.38627 -5.47112, 102.38682 ..."
Gorontalo,"MULTIPOLYGON (((122.12139 0.40583, 122.12167 0..."
Papbar,"MULTIPOLYGON (((133.50668 -4.32504, 133.50676 ..."
Jakarta,"MULTIPOLYGON (((106.73338 -6.09776, 106.73373 ..."
Jambi,"MULTIPOLYGON (((104.21628 -1.06326, 104.21578 ..."
